In [1]:
from nltk.tokenize import sent_tokenize
import pandas as pd
import sys
import time
import os



#sys.path.append("../../oats")
#from oats.pubmed.query import search, fetch_details
#from oats.utils.utils import save_to_pickle, load_from_pickle

In [4]:
from Bio import Entrez

def search(query,limit):
    Entrez.email = 'a@b.com'
    handle = Entrez.esearch(db='pubmed',sort='relevance',retmax=limit,retmode='xml',term=query)
    results = Entrez.read(handle)
    return results


def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'a@b.com'
    handle = Entrez.efetch(db='pubmed', retmode='xml', id=ids)
    results = Entrez.read(handle)
    return results

In [7]:
df = pd.read_csv("../../plant-data/genes_texts_annots.csv")


,id,species,unique_gene_identifiers,other_gene_identifiers,gene_models,descriptions,annotations,sources
0,0,zma,ns1|narrow sheath1|ns1-R|uniprot=Q70UV1|ncbi=5...,HB-type transcription factor|prs|WUSCHEL-relat...,GRMZM2G069028|Zm00001d004889,Plants (stems) do not elongate in response to ...,GO:0009739|PATO:0002302|PO:0000003|PATO:000056...,Plant PhenomeNET|MaizeGDB
1,1,zma,ns2|narrow sheath2|NM_001111772.1|ns2-R|unipro...,WUSCHEL-related homeobox 3B,Zm00001d052598,Plants (stems) do not elongate in response to ...,GO:0009739|PATO:0002302|PO:0000003|PATO:000056...,Plant PhenomeNET|MaizeGDB
2,2,ath,UBP26|Ubiquitin-Specific Protease|UBP15|At3g49...,ubiquitin-specific protease 15|F20D23.20|F20D2...,At3g49600|At1g17110|AT1G17110|AT3G49600,50% defective seeds. Low penetrance of endospe...,PO:0009010|PATO:0001241|PATO:0000460|GO:200001...,Plant PhenomeNET|TAIR
3,3,mtr,cyp716A12|cytochrome P450 monoxygenase|Medtr3g...,lha,Medtr3g021350|Medtr3g021180,Impaired hemolytic saponin synthesis. Plant gr...,GO:0016135|PATO:0001236|PATO:0000460|PO:000000...,Plant PhenomeNET
4,4,osa,SL1|STAMENLESS 1|Os01g0129200,stl1.1,Os01g0129200,Degenerated lemma. Degenerated palea. Stamen n...,PO:0009037|PATO:0000646|PO:0007130|PO:0009038|...,Plant PhenomeNET
...,...,...,...,...,...,...,...,...
6695,8148,ath,XAV,NaN,NaN,"Yellow seedling lethal, heterozygote light green.",GO:0008150|GO:0005575|GO:0003674,TAIR
6696,8149,ath,XTC1,NaN,NaN,First one or two leaves transformed into cotyl...,GO:0005575|GO:0003674,TAIR
6697,8150,ath,XTC2,NaN,NaN,First one or two leaves transformed into cotyl...,GO:0009793|GO:0005575|GO:0003674,TAIR
6698,8151,ath,XV,NaN,NaN,"Yellow-green cotyledons, leaf margins yellow, ...",GO:0005575|GO:0008150|GO:0003674,TAIR


In [ ]:
query = ""